In [17]:
import numpy as np
import random
import math
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from xgboost import XGBRegressor

In [6]:
filename = "train.csv"
df = pd.read_csv(filename, nrows=300000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
299995,2015-03-17 10:39:36.0000004,4.5,2015-03-17 10:39:36 UTC,-74.002090,40.729954,-73.995422,40.739185,2
299996,2012-10-01 09:59:32.0000001,9.0,2012-10-01 09:59:32 UTC,-74.003177,40.747336,-73.983265,40.746194,1
299997,2012-12-10 21:59:00.000000197,11.0,2012-12-10 21:59:00 UTC,-73.988432,40.720217,-73.994832,40.746707,1
299998,2014-03-03 11:14:53.0000001,5.5,2014-03-03 11:14:53 UTC,-73.980647,40.721606,-73.994044,40.727628,1


In [7]:
df = df.drop(df[df['fare_amount']<=0].index,axis=0)
df = df.drop(df[df['pickup_latitude']>45].index,axis=0)
df = df.drop(df[df['pickup_latitude']<40].index,axis=0)
df = df.drop(df[df['pickup_longitude']>-71].index,axis=0)
df = df.drop(df[df['pickup_longitude']<-75].index,axis=0)
df = df.drop(df[df['dropoff_latitude']>45].index,axis=0)
df = df.drop(df[df['dropoff_latitude']<40].index,axis=0)
df = df.drop(df[df['dropoff_longitude']>-71].index,axis=0)
df = df.drop(df[df['dropoff_longitude']<-80].index,axis=0)
df = df.drop(df[df['passenger_count']==0].index,axis=0)
df = df.drop('key',axis=1)
df = df.drop('pickup_datetime',axis=1)
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,292739.000000,292739.000000,292739.000000,292739.000000,292739.000000,292739.000000
mean,11.354127,-73.975104,40.751098,-73.974115,40.751437,1.690970
std,9.875685,0.039802,0.031133,0.039650,0.034002,1.307736
min,0.010000,-74.968142,40.052722,-75.448579,40.041180,1.000000
25%,6.000000,-73.992268,40.736602,-73.991533,40.735605,1.000000
50%,8.500000,-73.982056,40.753464,-73.980560,40.753917,1.000000
75%,12.500000,-73.968322,40.767519,-73.965233,40.768388,2.000000
max,500.000000,-71.056408,43.183332,-71.038222,43.415190,6.000000


In [8]:
y = df['fare_amount']
y = np.array(y,dtype=float)
X = df.drop('fare_amount',axis=1)
X = np.array(X,dtype=float)

In [9]:
df = pd.read_csv('test.csv')
df = df.drop('pickup_datetime',axis=1)
key = df['key']
X_test = df.drop('key',axis=1)
X_test.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [10]:
regr = KNeighborsRegressor(n_neighbors=7)
regr.fit(X,y)

KNeighborsRegressor(n_neighbors=7)

In [11]:
y_pred = regr.predict(X_test)
data = {'key':key,'fare_amount':y_pred}
ans = pd.DataFrame(data)
ans.to_csv('KNNsub.csv',index=False)
ans

/home/cmaspi/.local/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(


,key,fare_amount
0,2015-01-27 13:08:24.0000002,8.200000
1,2015-01-27 13:08:24.0000003,9.471429
2,2011-10-08 11:53:44.0000002,4.728571
3,2012-12-01 21:12:12.0000002,7.471429
4,2012-12-01 21:12:12.0000003,17.757143
...,...,...
9909,2015-05-10 12:37:51.0000002,8.500000
9910,2015-01-12 17:05:51.0000001,9.400000
9911,2015-04-19 20:44:15.0000001,55.731429
9912,2015-01-31 01:05:19.0000005,16.485714


In [12]:
regr1 = RandomForestRegressor(n_estimators=200)
regr1.fit(X,y)

RandomForestRegressor(n_estimators=200)

In [13]:
y_pred = regr1.predict(X_test)
data = {'key':key,'fare_amount':y_pred}
ans = pd.DataFrame(data)
ans.to_csv('RFsub.csv',index=False)
ans

/home/cmaspi/.local/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


,key,fare_amount
0,2015-01-27 13:08:24.0000002,8.35750
1,2015-01-27 13:08:24.0000003,8.76250
2,2011-10-08 11:53:44.0000002,5.27250
3,2012-12-01 21:12:12.0000002,7.63850
4,2012-12-01 21:12:12.0000003,16.63150
...,...,...
9909,2015-05-10 12:37:51.0000002,8.70650
9910,2015-01-12 17:05:51.0000001,10.01250
9911,2015-04-19 20:44:15.0000001,53.72570
9912,2015-01-31 01:05:19.0000005,15.36900


In [14]:
regr2 = GradientBoostingRegressor(n_estimators=1000)
regr2.fit(X,y)

GradientBoostingRegressor(n_estimators=1000)

In [15]:
y_pred = regr2.predict(X_test)
data = {'key':key,'fare_amount':y_pred}
ans = pd.DataFrame(data)
ans.to_csv('GBsub.csv',index=False)
ans

/home/cmaspi/.local/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


,key,fare_amount
0,2015-01-27 13:08:24.0000002,7.792067
1,2015-01-27 13:08:24.0000003,8.033543
2,2011-10-08 11:53:44.0000002,6.358389
3,2012-12-01 21:12:12.0000002,7.899501
4,2012-12-01 21:12:12.0000003,14.972845
...,...,...
9909,2015-05-10 12:37:51.0000002,8.695693
9910,2015-01-12 17:05:51.0000001,8.463254
9911,2015-04-19 20:44:15.0000001,49.981487
9912,2015-01-31 01:05:19.0000005,20.445963


In [18]:
regr3 = XGBRegressor()
regr3.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [19]:
y_pred = regr3.predict(X_test)
data = {'key':key,'fare_amount':y_pred}
ans = pd.DataFrame(data)
ans.to_csv('XGBsub.csv',index=False)
ans

,key,fare_amount
0,2015-01-27 13:08:24.0000002,7.394042
1,2015-01-27 13:08:24.0000003,7.985307
2,2011-10-08 11:53:44.0000002,6.190505
3,2012-12-01 21:12:12.0000002,8.019666
4,2012-12-01 21:12:12.0000003,16.240841
...,...,...
9909,2015-05-10 12:37:51.0000002,8.990083
9910,2015-01-12 17:05:51.0000001,9.158989
9911,2015-04-19 20:44:15.0000001,53.553581
9912,2015-01-31 01:05:19.0000005,16.433533
